In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-nov-2021/train.csv
/kaggle/input/tabular-playground-series-nov-2021/test.csv


# **Data research**

In [2]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-nov-2021/test.csv")
train.head()

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,target
0,0,0.106643,3.59437,132.8040,3.18428,0.081971,1.18859,3.73238,2.266270,2.09959,...,1.09862,0.013331,-0.011715,0.052759,0.065400,4.211250,1.97877,0.085974,0.240496,0
1,1,0.125021,1.67336,76.5336,3.37825,0.099400,5.09366,1.27562,-0.471318,4.54594,...,3.46017,0.017054,0.124863,0.154064,0.606848,-0.267928,2.57786,-0.020877,0.024719,0
2,2,0.036330,1.49747,233.5460,2.19435,0.026914,3.12694,5.05687,3.849460,1.80187,...,4.88300,0.085222,0.032396,0.116092,-0.001688,-0.520069,2.14112,0.124464,0.148209,0
3,3,-0.014077,0.24600,779.9670,1.89064,0.006948,1.53112,2.69800,4.517330,4.50332,...,3.47439,-0.017103,-0.008100,0.062013,0.041193,0.511657,1.96860,0.040017,0.044873,0
4,4,-0.003259,3.71542,156.1280,2.14772,0.018284,2.09859,4.15492,-0.038236,3.37145,...,1.91059,-0.042943,0.105616,0.125072,0.037509,1.043790,1.07481,-0.012819,0.072798,1


In [3]:
train.shape # data dimension

(600000, 102)

In [4]:
train['target'].value_counts() #target variable

1    303606
0    296394
Name: target, dtype: int64

In [5]:
train.isna().sum()# check if there are gaps in the data

id        0
f0        0
f1        0
f2        0
f3        0
         ..
f96       0
f97       0
f98       0
f99       0
target    0
Length: 102, dtype: int64

In [6]:
train.dtypes

id          int64
f0        float64
f1        float64
f2        float64
f3        float64
           ...   
f96       float64
f97       float64
f98       float64
f99       float64
target      int64
Length: 102, dtype: object

# **Data preparation**

In [7]:
# Get train data without the target and ids
X = train.iloc[:, 1:-1].copy()
# Get the target
y = train.target.copy()

# Create test X, drop ids.
test_X = test.iloc[:, 1:].copy()

In [8]:
import gc 
import matplotlib.image as img
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse, r2_score as r2
from lightgbm import LGBMRegressor

In [9]:
# Plot
import seaborn as sns
import matplotlib.pyplot as plt

# Scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Cross-Validation
from sklearn.model_selection import StratifiedKFold

In [10]:
# Apply a scaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
test_X = scaler.transform(test_X)

# **Building a model**

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [12]:
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [13]:
network = models.Sequential()
network.add(layers.Dense(units=16, activation="relu", input_shape=(100,)))
network.add(layers.Dense(units=16, activation="relu"))
network.add(layers.Dense(units=1, activation="sigmoid"))


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

In [14]:
network.compile(
    loss="binary_crossentropy",
    optimizer="rmsprop",
    metrics=["accuracy"]
)

In [15]:
history = network.fit(
    X_train,
    y_train,
    epochs=11,
    verbose=1,
    batch_size=100,
    validation_data=(X_test, y_test))

2021-11-29 08:10:35.867918: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/11
4200/4200 [==============================] - 12s 3ms/step - loss: 0.6200 - accuracy: 0.6792 - val_loss: 0.6174 - val_accuracy: 0.6766
Epoch 2/11
4200/4200 [==============================] - 10s 2ms/step - loss: 0.6054 - accuracy: 0.6989 - val_loss: 0.6007 - val_accuracy: 0.7051
Epoch 3/11
4200/4200 [==============================] - 10s 2ms/step - loss: 0.5988 - accuracy: 0.7052 - val_loss: 0.5936 - val_accuracy: 0.7119
Epoch 4/11
4200/4200 [==============================] - 10s 2ms/step - loss: 0.5943 - accuracy: 0.7097 - val_loss: 0.5938 - val_accuracy: 0.7100
Epoch 5/11
4200/4200 [==============================] - 10s 2ms/step - loss: 0.5914 - accuracy: 0.7124 - val_loss: 0.5867 - val_accuracy: 0.7187
Epoch 6/11
4200/4200 [==============================] - 10s 2ms/step - loss: 0.5895 - accuracy: 0.7149 - val_loss: 0.5853 - val_accuracy: 0.7205
Epoch 7/11
4200/4200 [==============================] - 10s 2ms/step - loss: 0.5880 - accuracy: 0.7170 - val_loss: 0.5846 - val_ac

In [16]:
history = network.fit(
    X,
    y,
    epochs=11,
    verbose=1,
    batch_size=100,)

Epoch 1/11
6000/6000 [==============================] - 12s 2ms/step - loss: 0.5831 - accuracy: 0.7232
Epoch 2/11
6000/6000 [==============================] - 12s 2ms/step - loss: 0.5826 - accuracy: 0.7239
Epoch 3/11
6000/6000 [==============================] - 12s 2ms/step - loss: 0.5816 - accuracy: 0.7250
Epoch 4/11
6000/6000 [==============================] - 12s 2ms/step - loss: 0.5809 - accuracy: 0.7262
Epoch 5/11
6000/6000 [==============================] - 12s 2ms/step - loss: 0.5805 - accuracy: 0.7265
Epoch 6/11
6000/6000 [==============================] - 12s 2ms/step - loss: 0.5800 - accuracy: 0.7274
Epoch 7/11
6000/6000 [==============================] - 12s 2ms/step - loss: 0.5798 - accuracy: 0.7274
Epoch 8/11
6000/6000 [==============================] - 12s 2ms/step - loss: 0.5794 - accuracy: 0.7280
Epoch 9/11
6000/6000 [==============================] - 12s 2ms/step - loss: 0.5791 - accuracy: 0.7285
Epoch 10/11
6000/6000 [==============================] - 12s 2ms/step - l

In [17]:
y_pred = network.predict(test_X)

In [18]:
y_pred_df = pd.DataFrame(y_pred, columns=['target'])
d = {0: '0.1', 1: '1'}
y_pred_df['target'] = y_pred_df['target'].map(d).fillna(y_pred_df['target'])
y_pred_df = pd.Series(y_pred_df['target'])
y_pred_df = y_pred_df.reset_index()
y_pred_df['index']=y_pred_df['index']+600000
y_pred_df=y_pred_df.rename(columns={'index': 'id'})
y_pred_df

,id,target
0,600000,0.750988
1,600001,0.763004
2,600002,0.761015
3,600003,0.615668
4,600004,0.732286
...,...,...
539995,1139995,0.774485
539996,1139996,0.758644
539997,1139997,0.702762
539998,1139998,0.748338


In [19]:
y_pred_df.to_csv('submission.csv', index=False)